<div style="
  padding: 5pt;
  border-style: solid;
  border-width: 1px;
  border-color: gray;
  border-radius: 10px;">
  
# **Python et intelligence artificielle**

# *Séance n°11a : Perceptron multicouche fait maison (partie 1)*

</div>

Durant ces deux séances, nous allons continuer notre exploration de l'intelligence
artificielle avec la construction d'un **réseau de neurones artificiel** que nous
associerons à un problème d'apprentissage supervisé. Ce réseau de neurones multicouche
(appelé aussi **perceptron multicouche**) sera implémenté sans utiliser de bibliothèques
spécialisées telles que *Scikit-learn*, *Keras* ou *TensorFlow*. Ce type de réseau
constitue la base des réseaux de neurones profonds et il nous permettra de résoudre des
tâches de classification plus ou moins complexes.

---

### Objectifs

Au cours de ces deux séances, vous allez :

- implémenter pas à pas un perceptron multicouche sans utiliser de bibliothèques spécialisées (*Scikit-learn*, *Keras* ou *TensorFlow*) ;
- appliquer ce modèle au problème de classification binaire déjà abordé dans la [séance n°9](seance_09.ipynb) ;
- comparer la performance de ce perceptron fait maison avec celui implémenté dans *Scikit-learn*.

---

## Introduction

Les réseaux de neurones artificiels sont des modèles inspirés des réseaux neuronaux
biologiques. Ils sont adaptés à l'apprentissage des relations complexes dans les données
pour y reconnaître ou générer des motifs. Ces réseaux utilisent des unités de calcul
appelées **neurones** et reliées par des **poids** ajustables. Ces neurones sont
organisés en **couches** successives, chacune jouant un rôle différent dans le traitement
des données.

### Perceptron

Le modèle du **perceptron** proposé par [Rosenblatt](https://www.ling.upenn.edu/courses/cogs501/Rosenblatt1958.pdf)
en 1958 a été l'un des premiers exemples de neurone artificiel. Bien que le perceptron
simple soit limité aux problèmes de classification linéaire, l'ajout de couches cachées
et de fonctions d'activation non linéaires a donné naissance au **perceptron multicouche** (MLP),
capable de traiter des problèmes plus complexes et non linéaires.

### Structure et fonctionnement d'un perceptron multicouche

Un perceptron multicouche se compose de trois types de couches :

1. Une **couche d'entrée** dans laquelle chaque neurone représente une caractéristique des données d'entrée.
2. Une ou plusieurs **couches cachées** qui permettent l'apprentissage de relations non linéaires entre les caractéristiques des données.
3. Une **couche de sortie** chargée de la prédiction finale du réseau.

Dans chaque neurone, les données d'entrée ($x_1, x_2, \cdots, x_n$) sont pondérées
par des **poids** ($w_1, w_2, \cdots, w_n$), et un **biais** $b$ est ajouté à la
somme des entrées pondérées. Ce résultat est ensuite transformé par une
**fonction d'activation** $\phi$ de manière à générer la sortie $y$ du neurone :

$$
y = \phi(z)
$$

où $z$ est la somme pondérée définie par :
$$
z = \sum_{i=1}^{n} w_i x_i + b
$$

### Fonction d'activation

La fonction d'activation $\phi$ qui agit à la sortie pondérée d'un neurone
permet d'introduire une non-linéarité dans le modèle. Cette non-linéarité est
essentielle pour permettre au réseau l'apprentissage des relations complexes
entre les données. Voici quelques fonctions d'activation couramment utilisées :

| Fonction d'activation | Formule | Avantages |
|-----------------------|---------|-----------|
| **sigmoïde**          | $\phi(z) = \frac{1}{1 + e^{-z}}$ | Idéale pour la classification binaire, fournit une valeur entre 0 et 1 |
| **tanh**              | $\phi(z) = \tanh(z)$ | Produit des sorties entre -1 et 1, avec une sensibilité élevée aux petites variations |
| **ReLU**              | $\phi(z) = \max(0, z)$ | Rapide à calculer, elle facilite la convergence dans les réseaux profonds |
| **seuil**             | $\phi(z) = \begin{cases} 1 & \text{si } z \geq 0 \\ 0 & \text{sinon} \end{cases}$ | Simple, mais limitée aux frontières linéaires |

---

## Travail demandé

Nous allons construire progressivement notre réseau de neurones en réalisant les étapes suivantes :

1. **Préparation des données** : importation, encodage et standardisation des données.
2. **Implémentation d'un neurone** : construction d'une unité neuronale avec sa fonction d'activation.
3. **Construction d'une couche de neurones** : création d'une structure en couches.
4. **Assemblage du réseau de neurones** : création d'un perceptron multicouche.
5. **Entraînement avec rétropropagation** : introduction de l'algorithme de rétropropagation pour ajuster les poids et optimiser le modèle.
6. **Améliorations et ajustements** : mise en oeuvre de la régularisation L2.
7. **Comparaison des performances** : évaluation et comparaison des résultats obtenus avec ceux du modèle proposé par *Scikit-learn*.

### 1. Préparation des données

Tous les systèmes d'apprentissage automatique nécessitent une préparation des données
soignée afin de garantir leur efficacité, et les réseaux de neurones n'y font pas
exception. Les étapes de cette préparation incluent généralement :

1. L'**importation** des données.
2. L'**encodage** de la cible en valeurs numériques si cela est nécessaire.
3. La **division** des données en un ensemble d'entraînement et un ensemble de tests.
4. La **standardisation** ou la **normalisation** des données pour que la distribution soit uniforme et facilite la convergence.

Le code suivant réalise ces différentes opérations sur le jeu de données "`sonar.csv`" que nous avons déjà rencontré lors de la séance n°9.

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Importation des données avec Pandas
data = pd.read_csv('ressources/sonar.csv')

# Préparation des données
X = data.iloc[:, :-1].values  # 60 caractéristiques d'entrée
y = data.iloc[:, -1].apply(lambda x: 1 if x == 'M' else 0).values  # Encodage de la cible en valeurs binaires

# Division des données en deux jeux : entraînement et test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardisation
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
```

</div>

Les données sont maintenant prêtes pour l'apprentissage.

### 2. Implémentation d'un neurone

Un perceptron multicouche est composé de plusieurs neurones, chacun effectuant une
opération de base : prendre plusieurs entrées, les pondérer, ajouter un biais, puis
appliquer une fonction d'activation.

#### Fonctionnement du neurone

Chaque neurone reçoit un vecteur d'entrées $X = [x_1, x_2, ..., x_n]$ dont il
calcule la somme pondérée, ajoute un biais, et enfin transmet le résultat à une
fonction d'activation pour produire une sortie entre 0 et 1 :

$$
z = \sum_{i=1}^{n} w_i x_i + b
$$

La fonction d'activation, par exemple la fonction sigmoïde, transforme cette somme
pondérée en une probabilité :

$$
\phi(z) = \frac{1}{1 + e^{-z}}
$$

Nous rappelons que la fonction sigmoïde est utile pour les problèmes de classification
binaire, car elle génère une sortie proche de 1 pour une classe et proche de 0 pour
l'autre, facilitant ainsi la prise de décision. Toutefois, l'utilisation d'un seul
neurone reste limitée. Celui-ci ne peut capturer qu'une frontière de décision linéaire
et ne peut pas résoudre des tâches nécessitant une complexité plus élevée.

#### Implémentation et test du neurone

La classe `Neurone` que nous allons implémenter représente un neurone individuel.
La classe est munie des attributs suivants :

- `poids` : un tableau des poids associés aux entrées du neurone ;
- `biais` : un scalaire représentant le biais ajouté à la somme pondérée des entrées ;
- `fonction_activation` : une chaîne de caractères qui spécifie la fonction d'activation à utiliser (`'sigmoïde'`, `'relu'`, etc.).

et des méthodes suivantes :

- `__init__(self, nbr_entrees, fonction_activation='sigmoïde')` : Initialise les poids, le biais et la fonction d'activation du neurone. Les poids sont initialisés avec des valeurs aléatoires comprises entre -0.1 et 0.1, le biais à une valeur aléatoire proche de 0. Quant à la fonction d'activation, elle est fixée par défaut avec la fonction sigmoïde.
- `_appliquer_activation(self, x)` : Applique la fonction d'activation définie (sigmoïde, ReLU, etc.) à la somme pondérée et retourne la valeur activée.
- `calculer_sortie(self, entrees)` : Calcule la somme pondérée des entrées, applique la fonction d'activation, et retourne la sortie du neurone.

1. Implémentez la classe `Neurone`.
2. Créez un neurone qui utilise une fonction d'activation sigmoïde et testez-le avec une des données d'entraînement comme l'indique le code suivant :

   <div style="
     padding: 5pt;
     border-style: dashed;
     border-width: 1px;
     border-color: gray;">

   ```python
   # Test d'un neurone avec une fonction d'activation sigmoïde
   neurone_test_sigmoïde = Neurone(X_train.shape[1], fonction_activation='sigmoïde')
   sortie_sigmoïde = neurone_test_sigmoïde.calculer_sortie(X_train[0])
   print("Sortie du neurone (sigmoïde) :", sortie_sigmoïde)
   ```

   </div>
3. Créez un deuxième neurone qui utilise cette fois-ci une fonction d'activation ReLU. Qu'observez-vous ?

#### Solution


In [ ]:
# Votre code ici


Dans ces deux premières étapes, nous avons préparé nos données et introduit le concept
de neurone individuel avec une fonction d'activation (sigmoïde, etc.). Nous allons
maintenant implémenter une **couche de neurones**, étape essentielle dans la création
de notre réseau de neurones multicouche.

### 3. Construction d'une couche de neurones

Une couche de neurones est une **collection de neurones** qui fonctionnent en parallèle.
En combinant les sorties de plusieurs neurones, la couche est capable de produire des
représentations plus diversifiées des données qu'elle n'en a en entrée.

Mathématiquement, pour une couche contenant $m$ neurones, chaque neurone $j$ produit
une sortie $Z_j$ qui est calculée comme suit :

$$
Z_j = \phi_j \left( \sum_{i=1}^{n} w_{ij} x_i + b_j \right), \quad \forall j \in [1, m]
$$

où :

- $x_i$ représente les entrées de la couche ;
- $w_{ij}$ est le poids de la connexion entre l'entrée $i$ et le neurone $j$ ;
- $b_j$ est le biais du neurone $j$ ;
- $\phi_j$ est la fonction d'activation appliquée par le neurone $j$.

Cette couche constitue un "bloc" de base dans un réseau de neurones. Pour réaliser cette
structure, nous allons implémenter la classe `Couche` qui représente notre ensemble de
neurones. Elle reçoit les entrées, les transmet à chaque neurone, puis collecte les
sorties des neurones en une seule sortie. La classe `Couche` est munie de l'attribut
suivant :

- `neurones` : une liste d'objets de type `Neurone` dans laquelle chaque neurone possède ses propres poids et biais.

et des méthodes suivantes :

- `__init__(self, nbr_entrees, nbr_neurones, fonction_activation='sigmoïde')` : Crée la liste des `nbr_neurones` neurones de la couche, chacun ayant un nombre d'entrées `nbr_entrees` et une fonction d'activation associée.
- `calculer_sorties(self, entrees)` : Passe les entrées à chaque neurone, collecte leurs sorties, et retourne la sortie de la couche sous forme de tableau (vous utiliserez `np.array()` pour réaliser cette opération).

#### Implémentation et test d'une couche de neurones

1. Implémentez la classe `Couche`.
2. Créez et testez une couche de 10 neurones à fonction d'activation sigmoïde comme l'indique le code suivant :

   <div style="
     padding: 5pt;
     border-style: dashed;
     border-width: 1px;
     border-color: gray;">

   ```python
   # Test de la classe couche
   couche_test = Couche(X_train.shape[1], 10, fonction_activation='sigmoïde')
   sorties_couche = couche_test.calculer_sorties(X_train[0])
   print("Sorties de la couche pour un exemple de données d'entraînement :", sorties_couche)
   ```

   </div>
3. Créez une couche avec 10 neurones à fonction d'activation ReLU et testez-la. Comparez les résultats avec ceux obtenus avec la couche précédente. Que remarquez-vous ?

Une seule couche reste cependant insuffisante pour capturer des relations complexes. Pour aller plus loin, nous ajouterons des **couches cachées** dans les étapes suivantes afin de construire notre réseau multicouche.

#### Solution


In [ ]:
# Votre code ici


### 4. Construction d'un réseau de neurones multicouche

Comme nous l'avons vu, un réseau de neurones multicouche (ou perceptron multicouche) est constitué d'une **couche d'entrée**, d'une ou plusieurs **couches cachées** et d'une **couche de sortie**. Chaque couche intermédiaire ou cachée permet d'extraire des caractéristiques de plus en plus abstraites des données d'entrée. C'est cette profondeur et cette capacité à capter des **relations complexes** entre les caractéristiques qui distinguent les réseaux de neurones multicouche des modèles linéaires de base.

Le réseau multicouche fonctionne en **propagation avant** (ou *forward propagation*), c'est-à-dire que les données passent d'une couche à la suivante, chaque couche calculant puis transmettant ses sorties jusqu'à la couche finale.

#### Architecture d'un réseau de neurones multicouche

1. **Couche d'entrée** : elle reçoit les données d'entrée et chaque caractéristique est reliée à un neurone de la première couche.
2. **Couches cachées** : ensemble de plusieurs couches de neurones interconnectés où chaque couche prend en entrée les sorties de la couche précédente. Chaque couche cachée aide à extraire des caractéristiques intermédiaires.
3. **Couche de sortie** : elle génère la prédiction du modèle. Dans notre cas (classification binaire), elle se compose d'un seul neurone avec une fonction d'activation sigmoïde, de manière à fournir une probabilité comprise entre 0 et 1.

#### Classe `ReseauDeNeurones`

La classe `ReseauDeNeurones` que nous allons implémenter est composée de plusieurs couches successives, en commençant par les couches cachées, puis en terminant par une couche de sortie. Cette classe orchestre la propagation des données d'une couche à l'autre. Elle est munie de l'attribut suivant :

- `couches` : une liste d'objets de type `Couche` qui constituent l'architecture du réseau. Chaque couche reçoit les sorties de la couche précédente.

et des méthodes suivantes :

- `__init__(self, nbr_entrees, structure_couches, fonction_activation='relu', fonction_activation_sortie='sigmoïde')` : Initialise le réseau avec les caractéristiques suivantes :
  - `nbr_entrees` spécifie le nombre de caractéristiques en entrée.
  - `structure_couches` est une liste où chaque élément indique le nombre de neurones dans une couche. Par exemple, `[30, 15, 1]` signifie deux couches cachées avec 30 neurones dans la première, 15 dans la suivante, et une couche de sortie avec 1 neurone.
  - `fonction_activation` est utilisée pour toutes les couches cachées. Par défaut, nous utilisons la fonction `relu` qui est couramment employée pour ses avantages en termes de convergence.
  - `fonction_activation_sortie` est la fonction d'activation appliquée à la couche de sortie. Ici, nous utilisons la fonction sigmoïde pour obtenir une sortie entre 0 et 1, compatible avec notre tâche de classification binaire.
- `calculer_sortie(self, entrees)` fait passer les entrées à travers chaque couche en calculant la propagation avant, et retourne la sortie finale du réseau.

#### Implémentation et test du réseau de neurones

1. Implémentez la classe `ReseauDeNeurones`.
2. Testez le réseau avec 2 couches cachées comprenant 30 neurones dans la première, 15 dans la suivante et suivie par une couche de sortie à 1 neurone.

   <div style="
     padding: 5pt;
     border-style: dashed;
     border-width: 1px;
     border-color: gray;">

   ```python
   # Test du réseau sur un échantillon de données
   reseau_test = ReseauDeNeurones(X_train.shape[1], [30, 15, 1])
   sortie_reseau = reseau_test.calculer_sortie(X_train[0])
   print("Sortie du réseau pour un échantillon des données d'entraînement :", sortie_reseau)
   ```

   </div>
3. Testez à nouveau le réseau en considérant une couche cachée de 10 neurones suivie d'une couche de sortie à 1 neurone. Observez comment cela affecte la sortie du réseau pour l'exemple d'entraînement précédent.
4. Testez d'autres configurations et décrivez leurs résultats.

#### Solution


In [ ]:
# Votre code ici


Vérifions avec la fonction d'évaluation suivante l'exactitude du réseau sur notre jeu de données :

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
def evaluer_reseau(reseau, X, y):
    y_pred = [reseau.calculer_sortie(x)[0] > 0.5 for x in X]  # Prédictions binaires
    return accuracy_score(y, y_pred)

# Évaluation sur l'ensemble de test
exactitude = evaluer_reseau(reseau_entrainement, X_test, y_test)
print("Exactitude du réseau sur l'ensemble de test :", exactitude)
```

</div>


In [ ]:
# Votre code ici


Nous avons maintenant un réseau capable de calculer une sortie, mais il manque un mécanisme d'apprentissage fondamental : celui qui permet au réseau de "remonter" ses erreurs de façon à ajuster les poids et les biais des neurones.

Dans la deuxième partie, nous implémenterons l'algorithme de **rétropropagation** qui va permettre en ajustant correctement les paramètres du réseau de minimiser l'erreur entre les sorties obtenues et les sorties attendues. L'objectif final est d'optimiser la capacité de généralisation et d'apprentissage de notre réseau sur les tâches les plus variées.

---

## Conclusion

Dans cette séance, vous avez :

- implémenté en partie un perceptron multicouche sans utiliser de bibliothèques spécialisées ;
- appliqué ce modèle incomplet à un problème de classification binaire.
